In [1]:
# Import
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from time import sleep

In [2]:
# Initialize Browser

#Firefox
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
options.headless = True
firefox_binary = FirefoxBinary()
browser = webdriver.Firefox(executable_path=r'C:\bin\geckodriver.exe', options=options, firefox_binary=firefox_binary)

# Chrome
#executable_path = {'executable_path': 'c:/bin/chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=True)

# Variables
player_table = []
current_year = '2023'
logging = 'off'

# Read csv
hitter_tables = pd.read_csv('dp_hitter_table.csv', encoding = "ISO-8859-1", header=0).to_dict('records')
pitcher_tables = pd.read_csv('dp_pitcher_table.csv', encoding = "ISO-8859-1", header=0).to_dict('records')
dp_list = pd.read_csv('input files/dp_salary_input.csv')
urls = dp_list['url']


In [3]:
# Loop for each player in dp_salary_input.csv
csv_loop_count = 0
for url in urls:

    # Player Link and DP Year
    fg_link = url
    
    # Firefox
    browser.get(fg_link)
    html = browser.page_source
    sleep(2)
    
    # Chrome
    #browser.visit(fg_link)
    #html = browser.html
    
    # Get Soup
    soup = bs(html, 'html.parser')
    
    # Get Player Name
    player_name = soup.select('#content > div > div.player-pages-layout_player-pages-layout__content__U2dUD > div:nth-child(1) > div.header_header__wrapper__4W49n > div > div.header_header__player-info__bLcjg > div.header_player-info__top__UwQPj > div.header_player-name__kZWhq > h1')[0].text
    position = soup.select('#content > div > div.player-pages-layout_player-pages-layout__content__U2dUD > div:nth-child(1) > div.header_header__wrapper__4W49n > div > div.header_header__player-info__bLcjg > div.header_player-info__top__UwQPj > div.header_item__6AFbi.header_item__position__tADVT.header_is-mobile__HHswX')[0].text
    if position == "PH/PR":
        position = "DH"
    

    #################################
    ####### P I T C H E R S #########
    #################################
    if 'P' in position:
    #if 'position=P' in fg_link:

        # Player Stats
        player_stats = soup.select('#standard > div:nth-child(3) > div > div > div > div.table-fixed')

        # Variables
        total_w = 0
        total_g = 0
        total_sv = 0
        total_hld = 0
        total_ip = 0
        total_h = 0
        total_er = 0
        total_bb = 0
        total_so = 0
        dp_modifier = 0
        final_position = ""
        triggered = []

        for div in player_stats:

            # For each row of data that has MLB stats
            for mlb_stats in div.findAll('tr', {'class': 'row-mlb-season'}):
                
                
                # Create/clear yearly stats
                stats_year = mlb_stats.findAll('td', {'data-stat': 'Season'})[0].text
                dp_ip = 0
                
                # Pull Stats
                ws = mlb_stats.findAll('td', {'data-stat': 'W'})[0].text
                gs = mlb_stats.findAll('td', {'data-stat': 'G'})[0].text
                svs = mlb_stats.findAll('td', {'data-stat': 'SV'})[0].text
                hlds = mlb_stats.findAll('td', {'data-stat': 'HLD'})[0].text
                ips = mlb_stats.findAll('td', {'data-stat': 'IP'})[0].text
                hits = mlb_stats.findAll('td', {'data-stat': 'H'})[0].text
                ers = mlb_stats.findAll('td', {'data-stat': 'ER'})[0].text
                bbs = mlb_stats.findAll('td', {'data-stat': 'BB'})[0].text
                ks = mlb_stats.findAll('td', {'data-stat': 'SO'})[0].text
                
                # Detect DP Year
                if not triggered:
                    if float(ips) >= 50:
                        triggered.append(stats_year)
                else:
                    if float(ips) > 0:
                        triggered.append(stats_year)

                # Add totals for last year
                if stats_year == current_year:

                    total_w += int(ws)
                    total_g += int(gs)           
                    total_sv += int(svs)
                    total_hld += int(hlds)           
                    total_ip += float(ips)
                    total_h += int(hits)
                    total_er += int(ers)
                    total_bb += int(bbs)
                    total_so += int(ks)
                    
                    # Pitcher DP Modifier
                    full_ip = str(total_ip).split('.')[0]
                    fractional_ip = str(total_ip).split('.')[1]
                    if fractional_ip == '0':
                        dp_ip = float(full_ip)
                    if fractional_ip == '1':
                        dp_ip = float(full_ip) + .34
                    if fractional_ip == '2':
                        dp_ip = float(full_ip) + .67                    
                    dp_modifier = dp_ip / 200
                    
        # DP Year
        dp_year = len(triggered)
        #if dp_year > 6:
        #    dp_year = "Vet"
        
        # Rate Stats
        # ERA
        try:
            era = round(9 *  total_er / total_ip, 2)
        except:
            era = 100

        #WHIP
        try:
            whip = round((total_bb + total_h) / total_ip, 2)
        except:
            whip = 100

        # IP per appearance
        try:
            ip_per_app = total_ip / total_g
        except:
            ip_per_app = 0

        # Determine position for DP charts
        if ip_per_app < 3:
            final_position = 'RP'
        else:
            final_position = 'SP'    

        player_stats_dict = {
            'pos': final_position,
            'w': total_w,
            'g': total_g,
            'sv': total_sv,
            'hld': total_hld,
            'ip': total_ip,
            'so': total_so,
            'era': era,
            'whip': whip
        }

        # Salary Processing
        sal_k = 0
        sal_w = 0
        sal_era = 0
        sal_whip = 0
        sal_sv = 0
        sal_hld = 0

        for row in pitcher_tables:

            # W Salary
            if (total_w >= row['w_low']) & (total_w <= row['w_high']):
                sal_w += row['salary']

            # SV Salary
            if (total_sv >= row['sv_low']) & (total_sv <= row['sv_high']):
                sal_sv += row['salary']

            # HLD Salary
            if (total_hld >= row['hld_low']) & (total_hld <= row['hld_high']):
                sal_hld += row['salary']

            # RP
            if final_position == 'RP':

                # K Salary
                if (total_so >= row['rp_k_low']) & (total_so < row['rp_k_high']):
                    sal_k += (row['salary'] * dp_modifier)

                # ERA Salary
                if (era >= row['rp_era_low']) & (era < row['rp_era_high']):
                    sal_era += (row['salary'] * dp_modifier)

                # WHIP Salary
                if (whip >= row['rp_whip_low']) & (whip < row['rp_whip_high']):
                    sal_whip += (row['salary'] * dp_modifier)

            # SP
            if final_position == 'SP':

                # K Salary
                if (total_so >= row['sp_k_low']) & (total_so < row['sp_k_high']):
                    sal_k += (row['salary'] * dp_modifier)

                # ERA Salary
                if (era >= row['sp_era_low']) & (era < row['sp_era_high']):
                    sal_era += (row['salary'] * dp_modifier)

                # WHIP Salary
                if (whip >= row['sp_whip_low']) & (whip < row['sp_whip_high']):
                    sal_whip += (row['salary'] * dp_modifier)

        total_value = sal_k + sal_era + sal_whip + sal_sv + sal_hld + sal_w

        # Final Salary - Apply DP rates
        final_salary = 0
        if dp_year == 0:
            final_salary = 0       
        if dp_year == 1:
            final_salary = 0.3
        elif dp_year == 2:
            final_salary = 0.5
        elif dp_year == 3:
            final_salary = total_value * 0.3
        elif dp_year == 4:
            final_salary = total_value * 0.4
        elif dp_year == 5:
            final_salary = total_value * 0.6
        elif dp_year == 6:
            final_salary = total_value * 0.8
        elif dp_year > 6:
            final_salary = total_value

        ffs_valuation = total_value * 3
        
        player_table.append({
            "Player Name": player_name,
            "Total Value": round(total_value, 2),
            "Total Salary": round(final_salary,2),
            "FFS Valuation": ffs_valuation,
            "FG Link": fg_link,
            "Position": final_position,
            "W": total_w,
            "W Salary": sal_w,
            "SO": total_so,
            "SO Salary": sal_k,
            "ERA": era,
            "ERA Salary": sal_era,
            "WHIP": whip,
            "WHIP Salary": sal_whip,
            "SV": total_sv,
            "SV Salary": sal_sv,
            "HLD": total_hld,
            "HLD Salary": sal_hld,
            "Pitching Modifier": dp_modifier,
            "DP Year": dp_year
        })
        
        print(f'{player_name}')
        
        if logging == 'on':                
            print(f'POS: {final_position}')
            print(f'W({total_w}): ${sal_w}')
            print(f'SO({total_so}): ${sal_k}')
            print(f'ERA({era}): ${sal_era}')
            print(f'WHIP({whip}): ${sal_whip}')
            print(f'SV({total_sv}): ${sal_sv}')
            print(f'HLD({total_hld}): ${sal_hld}')
            print(f'Total Value: ${total_value}')
            print(f'Total Salary: ${round(final_salary, 2)}')
            print(f'Projected FFS Valuation: ${ffs_valuation}')
            print(f'Pitching Modifier: {dp_modifier}')
            print(f'DP Year: {dp_year}')
            print('')


    #################################
    ######### H I T T E R S #########
    #################################
    if 'P' not in position:
    #if 'position=P' not in fg_link:

        # Player Positions
        player_header = soup.findAll('div', {'class': 'player-page-table__header'})

        # Variables
        positions = []
        gp_by_pos = []

        # Fangraphs Player Header
        for div in player_header:

            # Set count to skip first th row ('<Year> Games Played')
            count = 0

            # For each table header
            for th in div.findAll('th'):

                # If entry is not the first th
                if count > 0:
                    pos = th.text
                    pos = pos.replace(':', '').strip()
                    positions.append(pos)

                # Increment count
                count += 1

            # For each table data
            for td in div.findAll('td'):

                gp = td.text
                gp_by_pos.append(gp)

        player_pos = {
            "position": positions,
            "games_played": gp_by_pos
        }

        # Player Stats
        player_stats = soup.select('#standard > div:nth-child(3) > div > div > div > div.table-fixed')
        
        # Variables
        total_abs = 0
        total_hits = 0
        total_1b = 0
        total_2b = 0
        total_3b = 0
        total_runs = 0
        total_hrs = 0
        total_rbi = 0
        total_bb = 0
        total_hbp = 0
        total_sf = 0
        total_sb = 0
        triggered = []        

        # Fangraphs Player Stats (Standard) Table
        for div in player_stats:

            # For each row of data that has MLB stats
            for mlb_stats in div.findAll('tr', {'class': 'row-mlb-season'}):

                # Create/clear yearly stats
                stats_year = mlb_stats.findAll('td', {'data-stat': 'Season'})[0].text
                
                # Pull Stats
                atbats = mlb_stats.findAll('td', {'data-stat': 'AB'})[0].text
                hits = mlb_stats.findAll('td', {'data-stat': 'H'})[0].text
                singles = mlb_stats.findAll('td', {'data-stat': '1B'})[0].text
                doubles = mlb_stats.findAll('td', {'data-stat': '2B'})[0].text
                triples = mlb_stats.findAll('td', {'data-stat': '3B'})[0].text
                runs = mlb_stats.findAll('td', {'data-stat': 'R'})[0].text
                homeruns = mlb_stats.findAll('td', {'data-stat': 'HR'})[0].text
                rbis = mlb_stats.findAll('td', {'data-stat': 'RBI'})[0].text
                bbs = mlb_stats.findAll('td', {'data-stat': 'BB'})[0].text
                hbps = mlb_stats.findAll('td', {'data-stat': 'HBP'})[0].text
                sfs = mlb_stats.findAll('td', {'data-stat': 'SF'})[0].text
                sbs = mlb_stats.findAll('td', {'data-stat': 'SB'})[0].text
                
                # Detect DP Year
                if not triggered:
                    if int(atbats) >= 150:
                        triggered.append(stats_year)
                else:
                    if int(atbats) > 0:
                        triggered.append(stats_year)
                          
                # Get totals for last years
                if stats_year == current_year:

                    total_abs += int(atbats)           
                    total_hits += int(hits)            
                    total_1b += int(singles)             
                    total_2b += int(doubles)  
                    total_3b += int(triples)  
                    total_runs += int(runs)
                    total_hrs += int(homeruns)
                    total_rbi += int(rbis)            
                    total_bb += int(bbs)
                    total_hbp += int(hbps)               
                    total_sf += int(sfs)               
                    total_sb += int(sbs)

        # DP Year
        dp_year = len(triggered)
        #if dp_year > 6:
        #    dp_year = "Vet"
                    
        # Batting Average
        try:
            avg = round(total_hits/total_abs, 3)
        except:
            avg = 0

        # OPS
        try:
            ops = round(((total_hits + total_bb + total_hbp) / (total_abs + total_bb + total_sf + total_hbp)) + (((total_1b) + (total_2b * 2) + (total_3b * 3) + (4 * total_hrs)) / total_abs), 3)
        except:
            ops = 0
            
        player_stats_dict = {
            'ab': total_abs,
            'h': total_hits,
            'r': total_runs,
            'hr': total_hrs,
            'rbi': total_rbi,
            'sb': total_sb,
            'avg': avg,
            'ops': ops
        }

        # Salary Processing
        sal_r = 0
        sal_hr = 0
        sal_rbi = 0
        sal_sb = 0
        sal_avg = 0
        sal_ops = 0

        for row in hitter_tables:

            # R Salary
            if (total_runs >= row['r_low']) & (total_runs <= row['r_high']):
                sal_r += row['salary']

            # HR Salary
            if (total_hrs >= row['hr_low']) & (total_hrs <= row['hr_high']):
                sal_hr += row['salary']

            # RBI Salary
            if (total_rbi >= row['rbi_low']) & (total_rbi <= row['rbi_high']):
                sal_rbi += row['salary']

            # SB Salary
            if (total_sb >= row['sb_low']) & (total_sb <= row['sb_high']):
                sal_sb += row['salary']

            # AVG Salary
            if total_abs > 149:
                if (avg > row['avg_low']) & (avg <= row['avg_high']):
                    sal_avg += row['salary']

            # OPS Salary
            if total_abs > 149:
                if (ops > row['ops_low']) & (ops <= row['ops_high']):
                    sal_ops += row['salary']

        total_value = sal_r + sal_hr + sal_rbi + sal_sb + sal_avg + sal_ops

        # Final Salary - Apply DP rates
        final_salary = 0
        if dp_year == 0:
            final_salary = 0
        if dp_year == 1:
            final_salary = 0.3
        elif dp_year == 2:
            final_salary = 0.5
        elif dp_year == 3:
            final_salary = total_value * 0.3
        elif dp_year == 4:
            final_salary = total_value * 0.4
        elif dp_year == 5:
            final_salary = total_value * 0.6
        elif dp_year == 6:
            final_salary = total_value * 0.8
        elif dp_year > 6:
            final_salary = total_value
            
        ffs_valuation = total_value * 3
        
        player_table.append({
            "Player Name": player_name,
            "Total Value": total_value,
            "Total Salary": round(final_salary,2),
            "FFS Valuation": ffs_valuation,
            "FG Link": fg_link,
            "R": total_runs,
            "R Salary": sal_r,
            "HR": total_hrs,
            "HR Salary": sal_hr,
            "RBI": total_rbi,
            "RBI Salary": sal_rbi,
            "SB": total_sb,
            "SB Salary": sal_sb,
            "AVG": avg,
            "AVG Salary": sal_avg,
            "OPS": ops,
            "OPS Salary": sal_ops,
            "DP Year": dp_year
        })
        
        print(f'{player_name}')
        
        if logging == 'on':        
            print(f'R({total_runs}): ${sal_r}')
            print(f'HR({total_hrs}): ${sal_hr}')
            print(f'RBI({total_rbi}): ${sal_rbi}')
            print(f'SB({total_sb}): ${sal_sb}')
            print(f'AVG({avg}): ${sal_avg}')
            print(f'OPS({ops}): ${sal_ops}')
            print(f'Total Value: ${total_value}')
            print(f'Total Salary: ${round(final_salary,2)}')
            print(f'Projected FFS Valuation: ${ffs_valuation}')
            print(f'DP Year: {dp_year}')
            print('')
    
    csv_loop_count += 1
    sleep(5)

LaMonte Wade Jr.


In [4]:
pd.set_option("display.max_columns", None)
player_df = pd.DataFrame(player_table)
player_df
player_df.to_csv (f"output files/dps.csv", index = False, header=True)

In [5]:
hitter_tables

[{'salary': 0.5,
  'r_low': 51,
  'r_high': 58,
  'hr_low': 13,
  'hr_high': 15,
  'rbi_low': 46,
  'rbi_high': 55,
  'sb_low': 5,
  'sb_high': 5,
  'avg_low': 0.25,
  'avg_high': 0.258,
  'ops_low': 0.725,
  'ops_high': 0.761},
 {'salary': 1.0,
  'r_low': 59,
  'r_high': 74,
  'hr_low': 16,
  'hr_high': 21,
  'rbi_low': 56,
  'rbi_high': 73,
  'sb_low': 6,
  'sb_high': 13,
  'avg_low': 0.258,
  'avg_high': 0.271,
  'ops_low': 0.761,
  'ops_high': 0.809},
 {'salary': 1.5,
  'r_low': 75,
  'r_high': 83,
  'hr_low': 22,
  'hr_high': 24,
  'rbi_low': 74,
  'rbi_high': 81,
  'sb_low': 14,
  'sb_high': 18,
  'avg_low': 0.271,
  'avg_high': 0.278,
  'ops_low': 0.809,
  'ops_high': 0.829},
 {'salary': 2.0,
  'r_low': 84,
  'r_high': 93,
  'hr_low': 25,
  'hr_high': 29,
  'rbi_low': 82,
  'rbi_high': 95,
  'sb_low': 19,
  'sb_high': 23,
  'avg_low': 0.278,
  'avg_high': 0.289,
  'ops_low': 0.829,
  'ops_high': 0.858},
 {'salary': 2.5,
  'r_low': 94,
  'r_high': 122,
  'hr_low': 30,
  'hr_high'